In [7]:
import pandas as pd
import numpy as np
from sklearn.ensemble import RandomForestRegressor

In [107]:
df = pd.read_csv("02_Test_Data_Set/TV.csv")
df.head(3)

,Product_Name,Stars,Ratings,Reviews,current_price,MRP,channel,Operating_system,Picture_quality,Speaker,Frequency
0,Croma,4.2,1773,217,7990,20000,HD Ready 1366 x 768 Pixels,20 Speaker Output,60 Hz Refresh Rate,2 x HDMI | 2 x USB,1 Year Warranty
1,Adsun,3.8,6742,930,8699,21999,Netflix|Disney+Hotstar|Youtube,Operating System: Android Based,HD Ready 1366 x 768 Pixels,20 W Speaker Output,60 Hz Refresh Rate
2,LG,4.4,38870,3443,16499,21990,Netflix|Prime Video|Disney+Hotstar|Youtube,Operating System: WebOS,HD Ready 1366 x 768 Pixels,10 W Speaker Output,50 Hz Refresh Rate


## Q1.

In [81]:
df_sub1 = df[["Picture_quality", "Speaker", "Frequency"]].reset_index(drop = True)
df_sub1.head(10)

,Picture_quality,Speaker,Frequency
0,60 Hz Refresh Rate,2 x HDMI | 2 x USB,1 Year Warranty
1,HD Ready 1366 x 768 Pixels,20 W Speaker Output,60 Hz Refresh Rate
2,HD Ready 1366 x 768 Pixels,10 W Speaker Output,50 Hz Refresh Rate
3,HD Ready 1366 x 768 Pixels,20 W Speaker Output,60 Hz Refresh Rate
4,HD Ready 1366 x 768 Pixels,20 W Speaker Output,60 Hz Refresh Rate
5,HD Ready 1366 x 768 Pixels,20 W Speaker Output,60 Hz Refresh Rate
6,HD Ready 1366 x 768 Pixels,20 W Speaker Output,60 Hz Refresh Rate
7,HD Ready 1366 x 768 Pixels,20 Speaker Output,60 Hz Refresh Rate
8,Ultra HD (4K) 3840 x 2160 Pixels,20 W Speaker Output,60 Hz Refresh Rate
9,Full HD 1920 x 1080 Pixels,20 W Speaker Output,60 Hz Refresh Rate


In [82]:
ser_u = pd.Series(df_sub1["Picture_quality"].unique())
ser_u = pd.Series(df_sub1["Speaker"].unique())
ser_u = pd.Series(df_sub1["Frequency"].unique())
ser_u[ser_u.str.contains("Hz")]
# ser_u[ser_u.str.contains("[0-9]{2,3}Hz")]

1      60 Hz Refresh Rate
2      50 Hz Refresh Rate
15    120 Hz Refresh Rate
22    100 Hz Refresh Rate
24    800 Hz Refresh Rate
25    200 Hz Refresh Rate
dtype: object

In [83]:
# 60 Hz로 시작하거나 숫자(0~9)가 아닌 문자에 바로 이어서 60 Hz가 나오는 경우 (160 Hz, 260 Hz 등 필터링)
# ^ : 그냥 나오면 앞에 아무것도 없음, []안에 들어가면 뒤에 딸려오는 조건을 반전시킨다(여집합)
df_sub1["Hz_1"] = df_sub1["Picture_quality"].str.contains("^60 Hz|[^0-9]60 Hz", regex = True) + 0
df_sub1["Hz_2"] = df_sub1["Speaker"].str.contains("^60 Hz|[^0-9]60 Hz", regex = True) + 0
df_sub1["Hz_3"] = df_sub1["Frequency"].str.contains("^60 Hz|[^0-9]60 Hz", regex = True) + 0
df_sub1.head()

,Picture_quality,Speaker,Frequency,Hz_1,Hz_2,Hz_3
0,60 Hz Refresh Rate,2 x HDMI | 2 x USB,1 Year Warranty,1,0,0
1,HD Ready 1366 x 768 Pixels,20 W Speaker Output,60 Hz Refresh Rate,0,0,1
2,HD Ready 1366 x 768 Pixels,10 W Speaker Output,50 Hz Refresh Rate,0,0,0
3,HD Ready 1366 x 768 Pixels,20 W Speaker Output,60 Hz Refresh Rate,0,0,1
4,HD Ready 1366 x 768 Pixels,20 W Speaker Output,60 Hz Refresh Rate,0,0,1


In [87]:
df_sub1[["Hz_1", "Hz_2", "Hz_3"]].sum()

Hz_1     77
Hz_2     10
Hz_3    423
dtype: int64

In [89]:
df_sub1["Hz_cnt"] = df_sub1[["Hz_1", "Hz_2", "Hz_3"]].sum(axis = 1)
df_sub1.head()

,Picture_quality,Speaker,Frequency,Hz_1,Hz_2,Hz_3,Hz_cnt
0,60 Hz Refresh Rate,2 x HDMI | 2 x USB,1 Year Warranty,1,0,0,1
1,HD Ready 1366 x 768 Pixels,20 W Speaker Output,60 Hz Refresh Rate,0,0,1,1
2,HD Ready 1366 x 768 Pixels,10 W Speaker Output,50 Hz Refresh Rate,0,0,0,0
3,HD Ready 1366 x 768 Pixels,20 W Speaker Output,60 Hz Refresh Rate,0,0,1,1
4,HD Ready 1366 x 768 Pixels,20 W Speaker Output,60 Hz Refresh Rate,0,0,1,1


In [90]:
df_sub1["Hz_cnt"].value_counts()

1    510
0    156
Name: Hz_cnt, dtype: int64

In [98]:
# lambda 사용한 풀이
# df_sub1.iloc[:, :3].apply(lambda x : x.str.contains("60 Hz").sum(), axis = 1).sum()
df_sub1.iloc[:, :3].apply(lambda x : x.str.contains("60 Hz").sum(), axis = 1).value_counts()

1    510
0    156
dtype: int64

In [99]:
# extract 사용한 풀이
df_sub1["Hz_1"] = df_sub1["Picture_quality"].str.extract("([0-9]{2,3}) Hz").fillna(0).astype("int")
df_sub1["Hz_2"] = df_sub1["Speaker"].str.extract("([0-9]{2,3}) Hz").fillna(0).astype("int")
df_sub1["Hz_3"] = df_sub1["Frequency"].str.extract("([0-9]{2,3}) Hz").fillna(0).astype("int")
df_sub1.head()

,Picture_quality,Speaker,Frequency,Hz_1,Hz_2,Hz_3,Hz_cnt
0,60 Hz Refresh Rate,2 x HDMI | 2 x USB,1 Year Warranty,60,0,0,1
1,HD Ready 1366 x 768 Pixels,20 W Speaker Output,60 Hz Refresh Rate,0,0,60,1
2,HD Ready 1366 x 768 Pixels,10 W Speaker Output,50 Hz Refresh Rate,0,0,50,0
3,HD Ready 1366 x 768 Pixels,20 W Speaker Output,60 Hz Refresh Rate,0,0,60,1
4,HD Ready 1366 x 768 Pixels,20 W Speaker Output,60 Hz Refresh Rate,0,0,60,1


In [16]:
df_sub1["Frequency"] = np.where(df_sub1["Picture_quality"].str.contains(" Hz "),
                           df_sub1["Picture_quality"],
                           df_sub1["Frequency"])
df_sub1.head(10)

/home/heesandv/miniconda3/envs/datascience/lib/python3.7/site-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until


,Picture_quality,Speaker,Frequency
0,60 Hz Refresh Rate,2 x HDMI | 2 x USB,60 Hz Refresh Rate
1,HD Ready 1366 x 768 Pixels,20 W Speaker Output,60 Hz Refresh Rate
2,HD Ready 1366 x 768 Pixels,10 W Speaker Output,50 Hz Refresh Rate
3,HD Ready 1366 x 768 Pixels,20 W Speaker Output,60 Hz Refresh Rate
4,HD Ready 1366 x 768 Pixels,20 W Speaker Output,60 Hz Refresh Rate
5,HD Ready 1366 x 768 Pixels,20 W Speaker Output,60 Hz Refresh Rate
6,HD Ready 1366 x 768 Pixels,20 W Speaker Output,60 Hz Refresh Rate
7,HD Ready 1366 x 768 Pixels,20 Speaker Output,60 Hz Refresh Rate
8,Ultra HD (4K) 3840 x 2160 Pixels,20 W Speaker Output,60 Hz Refresh Rate
9,Full HD 1920 x 1080 Pixels,20 W Speaker Output,60 Hz Refresh Rate


In [78]:
df_sub1[df_sub1["Speaker"].str.contains(" Hz ")]

,Picture_quality,Speaker,Frequency,Hz_1,Hz_2,Hz_3
0,60 Hz Refresh Rate,2 x HDMI | 2 x USB,1 Year Warranty,1,0,0
1,HD Ready 1366 x 768 Pixels,20 W Speaker Output,60 Hz Refresh Rate,0,0,1
2,HD Ready 1366 x 768 Pixels,10 W Speaker Output,50 Hz Refresh Rate,0,0,0
3,HD Ready 1366 x 768 Pixels,20 W Speaker Output,60 Hz Refresh Rate,0,0,1
4,HD Ready 1366 x 768 Pixels,20 W Speaker Output,60 Hz Refresh Rate,0,0,1


In [18]:
df_sub1["Frequency"] = np.where(df_sub1["Speaker"].str.contains(" Hz "),
                           df_sub1["Speaker"],
                           df_sub1["Frequency"])
df_sub1[df_sub1["Speaker"].str.contains(" Hz ")]

/home/heesandv/miniconda3/envs/datascience/lib/python3.7/site-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until


,Picture_quality,Speaker,Frequency
158,24W Speaker Output,60 Hz Refresh Rate,60 Hz Refresh Rate
196,24W Speaker Output,60 Hz Refresh Rate,60 Hz Refresh Rate
242,20 W Speaker Output,50 Hz Refresh Rate,50 Hz Refresh Rate
297,16 W Speaker Output,60 Hz Refresh Rate,60 Hz Refresh Rate
315,20 W Speaker Output,60 Hz Refresh Rate,60 Hz Refresh Rate
344,40 W Speaker Output,60 Hz Refresh Rate,60 Hz Refresh Rate
378,16W Speaker Output,60 Hz Refresh Rate,60 Hz Refresh Rate
393,40 W Speaker Output,200 Hz Refresh Rate,200 Hz Refresh Rate
398,16 W Speaker Output,60 Hz Refresh Rate,60 Hz Refresh Rate
419,20 W Speaker Output,50 Hz Refresh Rate,50 Hz Refresh Rate


In [20]:
df_sub1.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 666 entries, 0 to 665
Data columns (total 3 columns):
 #   Column           Non-Null Count  Dtype 
---  ------           --------------  ----- 
 0   Picture_quality  666 non-null    object
 1   Speaker          666 non-null    object
 2   Frequency        666 non-null    object
dtypes: object(3)
memory usage: 15.7+ KB


In [21]:
df_sub1.dropna()
df_sub1.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 666 entries, 0 to 665
Data columns (total 3 columns):
 #   Column           Non-Null Count  Dtype 
---  ------           --------------  ----- 
 0   Picture_quality  666 non-null    object
 1   Speaker          666 non-null    object
 2   Frequency        666 non-null    object
dtypes: object(3)
memory usage: 15.7+ KB


In [69]:
df_sub1[df_sub1["Frequency"].str.contains("60 Hz")].count()

Picture_quality    510
Speaker            510
Frequency          510
dtype: int64

## Q2.

In [60]:
df_sub2 = df[["Stars", "Picture_quality", "Operating_system", "channel"]]
df_sub2.head()

,Stars,Picture_quality,Operating_system,channel
0,4.2,60 Hz Refresh Rate,20 Speaker Output,HD Ready 1366 x 768 Pixels
1,3.8,HD Ready 1366 x 768 Pixels,Operating System: Android Based,Netflix|Disney+Hotstar|Youtube
2,4.4,HD Ready 1366 x 768 Pixels,Operating System: WebOS,Netflix|Prime Video|Disney+Hotstar|Youtube
3,4.3,HD Ready 1366 x 768 Pixels,Operating System: Android,Netflix|Prime Video|Disney+Hotstar|Youtube
4,4.3,HD Ready 1366 x 768 Pixels,Operating System: Android,Netflix|Prime Video|Disney+Hotstar|Youtube


In [103]:
ser_u = pd.Series(df_sub2["Operating_system"].unique())
ser_u = pd.Series(df_sub2["channel"         ].unique())
ser_u = pd.Series(df_sub2["Picture_quality" ].unique())
ser_u[ser_u.str.contains("4K|8K")]

2                      Ultra HD (4K) 3840 x 2160 Pixels
4     Ultra HD (4K) Crystal 4K FE UHD (3840 x 2160) ...
7                        Ultra HD (4K) 3840×2160 Pixels
9                      Ultra HD (4K) 3860 x 2160 Pixels
10                       Ultra HD (4K) 3840X2160 Pixels
12                     Ultra HD (4K) 3840 × 2160 Pixels
13                       Ultra HD (4K) 3840*2160 Pixels
14                       Ultra HD (4K) 3840x2160 Pixels
28                     Ultra HD (8K) 7680 x 4320 Pixels
34                     Ultra HD (4K) 3840 � 2160 Pixels
dtype: object

In [105]:
df_sub2["4K_1"] = df_sub2["Operating_system"].str.contains("4K") + 0
df_sub2["4K_2"] = df_sub2["channel"         ].str.contains("4K") + 0
df_sub2["4K_3"] = df_sub2["Picture_quality" ].str.contains("4K") + 0
df_sub2["4K_cnt"] = df_sub2[["4K_1", "4K_2", "4K_3"]].sum(axis = 1)

df_sub2["8K_1"] = df_sub2["Operating_system"].str.contains("8K") + 0
df_sub2["8K_2"] = df_sub2["channel"         ].str.contains("8K") + 0
df_sub2["8K_3"] = df_sub2["Picture_quality" ].str.contains("8K") + 0
df_sub2["8K_cnt"] = df_sub2[["8K_1", "8K_2", "8K_3"]].sum(axis = 1)

df_sub2.head(2)

,Stars,Picture_quality,Operating_system,channel,Resolution_grade,4K_1,4K_2,4K_3,4K_cnt,8K_1,8K_2,8K_3,8K_cnt
0,4.2,60 Hz Refresh Rate,20 Speaker Output,HD Ready 1366 x 768 Pixels,HD,0,0,0,0,0,0,0,0
1,3.8,HD Ready 1366 x 768 Pixels,Operating System: Android Based,Netflix|Disney+Hotstar|Youtube,HD,0,0,0,0,0,0,0,0


In [106]:
stat_4k = df_sub2.loc[df_sub2["4K_cnt"] > 0, "Stars"].mean()
stat_8k = df_sub2.loc[df_sub2["8K_cnt"] > 0, "Stars"].mean()
round(abs(stat_4k - stat_8k), 2)

0.38

In [44]:
df_sub2["Picture_quality"].str.contains("HD")

0      False
1       True
2       True
3       True
4       True
       ...  
661    False
662     True
663    False
664    False
665    False
Name: Picture_quality, Length: 666, dtype: bool

In [45]:
df_sub2["Picture_quality"] = np.where(df_sub2["Operating_system"].str.contains("HD"),
                           df_sub2["Operating_system"],
                           df_sub2["Picture_quality"])
df_sub2["Picture_quality"] = np.where(df_sub2["Operating_system"].str.contains("4K"),
                           df_sub2["Operating_system"],
                           df_sub2["Picture_quality"])
df_sub2["Picture_quality"] = np.where(df_sub2["Operating_system"].str.contains("8K"),
                           df_sub2["Operating_system"],
                           df_sub2["Picture_quality"])
df_sub2["Picture_quality"] = np.where(df_sub2["channel"].str.contains("HD"),
                           df_sub2["channel"],
                           df_sub2["Picture_quality"])
df_sub2["Picture_quality"] = np.where(df_sub2["channel"].str.contains("4K"),
                           df_sub2["channel"],
                           df_sub2["Picture_quality"])
df_sub2["Picture_quality"] = np.where(df_sub2["channel"].str.contains("8K"),
                           df_sub2["channel"],
                           df_sub2["Picture_quality"])
df_sub2.head(5)

/home/heesandv/miniconda3/envs/datascience/lib/python3.7/site-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until
/home/heesandv/miniconda3/envs/datascience/lib/python3.7/site-packages/ipykernel_launcher.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
/home/heesandv/miniconda3/envs/datascience/lib/python3.7/site-packages/ipykernel_launcher.py:9: SettingWithCopyWarning: 
A value is trying to be se

,Stars,Picture_quality,Operating_system,channel
0,4.2,HD Ready 1366 x 768 Pixels,20 Speaker Output,HD Ready 1366 x 768 Pixels
1,3.8,HD Ready 1366 x 768 Pixels,Operating System: Android Based,Netflix|Disney+Hotstar|Youtube
2,4.4,HD Ready 1366 x 768 Pixels,Operating System: WebOS,Netflix|Prime Video|Disney+Hotstar|Youtube
3,4.3,HD Ready 1366 x 768 Pixels,Operating System: Android,Netflix|Prime Video|Disney+Hotstar|Youtube
4,4.3,HD Ready 1366 x 768 Pixels,Operating System: Android,Netflix|Prime Video|Disney+Hotstar|Youtube


In [51]:
df_sub2["Picture_quality"].str.contains("HD").value_counts()

True    666
Name: Picture_quality, dtype: int64

In [62]:
df_sub2["Resolution_grade"] = np.where(df_sub2["Picture_quality"].str.contains("4K"),
                                       "4K", "HD")
df_sub2["Resolution_grade"] = np.where(df_sub2["Picture_quality"].str.contains("8K"),
                                       "8K", df_sub2["Resolution_grade"])
df_sub2["Resolution_grade"].value_counts()

/home/heesandv/miniconda3/envs/datascience/lib/python3.7/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
/home/heesandv/miniconda3/envs/datascience/lib/python3.7/site-packages/ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  after removing the cwd from sys.path.


HD    337
4K    328
8K      1
Name: Resolution_grade, dtype: int64

In [63]:
df_sub2.groupby("Resolution_grade")["Stars"].mean().reset_index().round(2)

,Resolution_grade,Stars
0,4K,3.22
1,8K,3.60
2,HD,3.09


In [66]:
round(abs(3.22 - 3.6), 2)

0.38

## Q3.

In [116]:
df_q3 = df.copy()
df_q3.head(2)

,Product_Name,Stars,Ratings,Reviews,current_price,MRP,channel,Operating_system,Picture_quality,Speaker,Frequency
0,Croma,4.2,1773,217,7990,20000,HD Ready 1366 x 768 Pixels,20 Speaker Output,60 Hz Refresh Rate,2 x HDMI | 2 x USB,1 Year Warranty
1,Adsun,3.8,6742,930,8699,21999,Netflix|Disney+Hotstar|Youtube,Operating System: Android Based,HD Ready 1366 x 768 Pixels,20 W Speaker Output,60 Hz Refresh Rate


In [117]:
df_q3 = df_q3.loc[~df_q3["channel"].str.contains("Pixel|Oper"), ] # 판다스에서 ~ 문자를 앞에 쓰면 boolean 반전시켜줌
len(df), len(df_q3)

(666, 548)

In [118]:
df_q3 = df_q3.reset_index(drop = True)

In [119]:
# df_q3["Review_ratio"] = df_q3["Reviews"] / df_q3["Ratings"]
# df_q3["Discount"] = df_q3["current_price"] / df_q3["MRP"]
# df_q3["Netflix"] = df_q3["channel"].str.contains("Netflix") + 0
# df_q3["Prime_Video"] = df_q3["channel"].str.contains("Prime_Video") + 0
# df_q3["is_high_q"] = df_q3["Picture_quality"].str.contains("4K|8K") + 0
df_q3["x1"] = df_q3["Reviews"] / df_q3["Ratings"]
df_q3["x2"] = df_q3["MRP"]
df_q3["x3"] = df_q3["current_price"] / df_q3["MRP"]
df_q3["x4"] = df_q3["channel"].str.contains("Netflix") + 0
df_q3["x5"] = df_q3["channel"].str.contains("Prime_Video") + 0
df_q3["x6"] = df_q3["Picture_quality"].str.contains("4K|8K") + 0

In [120]:
df_q3.head(2)

,Product_Name,Stars,Ratings,Reviews,current_price,MRP,channel,Operating_system,Picture_quality,Speaker,Frequency,x1,x2,x3,x4,x5,x6
0,Adsun,3.8,6742,930,8699,21999,Netflix|Disney+Hotstar|Youtube,Operating System: Android Based,HD Ready 1366 x 768 Pixels,20 W Speaker Output,60 Hz Refresh Rate,0.137941,21999,0.395427,1,0,0
1,LG,4.4,38870,3443,16499,21990,Netflix|Prime Video|Disney+Hotstar|Youtube,Operating System: WebOS,HD Ready 1366 x 768 Pixels,10 W Speaker Output,50 Hz Refresh Rate,0.088577,21990,0.750296,1,0,0


In [122]:
df_model = df_q3[["Stars", "x1", "x2", "x3", "x4", "x5", "x6"]]
df_model = df_model.dropna().reset_index(drop = True)
print(len(df_model))
df_model.isna().sum()

197


Stars    0
x1       0
x2       0
x3       0
x4       0
x5       0
x6       0
dtype: int64

In [123]:
model_rf = RandomForestRegressor(random_state=123)
model_rf.fit(X = df_model.drop("Stars", axis = 1),
             y = df_model["Stars"])

RandomForestRegressor(random_state=123)

In [127]:
model_rf.feature_importances_

array([0.31717126, 0.17487942, 0.46113958, 0.0273387 , 0.        ,
       0.01947103])

In [125]:
df_model.columns[1:][model_rf.feature_importances_.argmax()] # numpy 객체는 .argmax(), pandas 객체는 .idxmax()

'x3'

In [124]:
round(model_rf.feature_importances_.max(), 3)

0.461